In [3]:
import duckdb
import pandas as pd

# ------------------------------------------------
# 1. Connecting to the Database
# ------------------------------------------------
def connect_to_database(db_path="nl2sql_agent/database/tweets.duckdb"):
    """
    Connect to an existing DuckDB database and return the connection object.
    """
    return duckdb.connect(db_path)

# ------------------------------------------------
# 2. Table Overview
# ------------------------------------------------
def get_table_info(con, table_name="tweets"):
    """
    Returns a Pandas DataFrame with the list of columns, data types, and 
    an approximate count of distinct values in each column.
    """
    # Use DESCRIBE instead of PRAGMA show_columns
    schema_query = f"""
        DESCRIBE {table_name}
    """
    schema_info = con.execute(schema_query).fetchdf()
    
    # 2) Distinct counts for columns
    distinct_counts = {}
    for col in schema_info['column_name']:
        query = f"SELECT COUNT(DISTINCT {col}) FROM {table_name}"
        distinct_counts[col] = con.execute(query).fetchone()[0]
    
    schema_info['distinct_count'] = schema_info['column_name'].apply(lambda x: distinct_counts[x])
    return schema_info

def get_total_row_count(con, table_name="tweets"):
    """
    Returns the total number of rows in the specified table.
    """
    query = f"SELECT COUNT(*) FROM {table_name}"
    return con.execute(query).fetchone()[0]

# ------------------------------------------------
# 3. Missing Values / Null Analysis
# ------------------------------------------------
def get_null_counts(con, table_name="tweets"):
    """
    Returns a dictionary mapping each column name to the number of nulls/empties.
    """
    # Get columns using DESCRIBE instead of PRAGMA
    cols = con.execute(f"DESCRIBE {table_name}").fetchdf()['column_name'].tolist()
    
    # Rest of the function remains the same
    null_query_parts = []
    for col in cols:
        part = f"(COUNT(*) - COUNT({col})) as {col}_nulls"
        null_query_parts.append(part)
    null_query = "SELECT " + ",\n       ".join(null_query_parts) + f" FROM {table_name}"
    
    result = con.execute(null_query).fetchdf().iloc[0].to_dict()
    
    # Convert to a more friendly dictionary: {col: null_count}
    null_dict = {}
    for col in cols:
        col_null = f"{col}_nulls"
        null_dict[col] = result[col_null]
    
    return null_dict

def print_null_summary(con, table_name="tweets"):
    """
    Prints a summary of null counts and percentages for each column.
    """
    total_rows = get_total_row_count(con, table_name)
    null_dict = get_null_counts(con, table_name)
    
    print(f"\nTotal rows in {table_name}: {total_rows:,}\n")
    print("Null/Empty value counts (column : null_count (percentage)):")
    for col, count_null in null_dict.items():
        perc = (count_null / total_rows) * 100 if total_rows > 0 else 0
        print(f"{col:<20} {count_null:>10,} ({perc:>5.1f}%)")

# ------------------------------------------------
# 4. Basic Statistics (Numeric Columns)
# ------------------------------------------------
def get_basic_numeric_stats(con, table_name="tweets"):
    """
    Returns a Pandas DataFrame containing average, min, max, and standard deviation
    for numeric columns of interest: replyCount, retweetCount, likeCount, quoteCount, viewCount.
    """
    stats_query = f"""
    SELECT 
        AVG(replyCount) AS avg_reply,
        MIN(replyCount) AS min_reply,
        MAX(replyCount) AS max_reply,
        STDDEV_SAMP(replyCount) AS std_reply,
        
        AVG(retweetCount) AS avg_retweet,
        MIN(retweetCount) AS min_retweet,
        MAX(retweetCount) AS max_retweet,
        STDDEV_SAMP(retweetCount) AS std_retweet,
        
        AVG(likeCount) AS avg_like,
        MIN(likeCount) AS min_like,
        MAX(likeCount) AS max_like,
        STDDEV_SAMP(likeCount) AS std_like,
        
        AVG(quoteCount) AS avg_quote,
        MIN(quoteCount) AS min_quote,
        MAX(quoteCount) AS max_quote,
        STDDEV_SAMP(quoteCount) AS std_quote,
        
        AVG(viewCount) AS avg_view,
        MIN(viewCount) AS min_view,
        MAX(viewCount) AS max_view,
        STDDEV_SAMP(viewCount) AS std_view
    FROM {table_name}
    """
    return con.execute(stats_query).fetchdf()

def print_basic_numeric_stats(con, table_name="tweets"):
    """
    Prints the basic numeric stats in a nicely formatted way.
    """
    df_stats = get_basic_numeric_stats(con, table_name)
    row = df_stats.iloc[0]
    
    print("Basic Numeric Statistics:\n")
    print(f"Replies   - Avg: {row['avg_reply']:.2f}, Min: {row['min_reply']}, Max: {row['max_reply']}, Std: {row['std_reply']:.2f}")
    print(f"Retweets  - Avg: {row['avg_retweet']:.2f}, Min: {row['min_retweet']}, Max: {row['max_retweet']}, Std: {row['std_retweet']:.2f}")
    print(f"Likes     - Avg: {row['avg_like']:.2f}, Min: {row['min_like']}, Max: {row['max_like']}, Std: {row['std_like']:.2f}")
    print(f"Quotes    - Avg: {row['avg_quote']:.2f}, Min: {row['min_quote']}, Max: {row['max_quote']}, Std: {row['std_quote']:.2f}")
    print(f"Views     - Avg: {row['avg_view']:.2f}, Min: {row['min_view']}, Max: {row['max_view']}, Std: {row['std_view']:.2f}")

# ------------------------------------------------
# 5. Top Tweets and Top Users
# ------------------------------------------------
def get_top_tweets_by_metric(con, metric="likeCount", top_n=10, table_name="tweets"):
    """
    Returns top N tweets with the highest value in a given numeric metric (e.g. 'likeCount').
    Also returns the text and user columns for context.
    """
    query = f"""
    SELECT 
        id,
        user,
        text,
        {metric} AS metric_value
    FROM {table_name}
    ORDER BY metric_value DESC
    LIMIT {top_n}
    """
    return con.execute(query).fetchdf()

def get_top_users_by_tweet_count(con, top_n=10, table_name="tweets"):
    """
    Returns top N users who have produced the most tweets in the dataset.
    """
    # We assume 'user' column is a simple string with the username or user ID.
    query = f"""
    SELECT 
        user,
        COUNT(*) AS tweet_count
    FROM {table_name}
    WHERE user IS NOT NULL AND user <> ''
    GROUP BY user
    ORDER BY tweet_count DESC
    LIMIT {top_n}
    """
    return con.execute(query).fetchdf()

# ------------------------------------------------
# 6. Hashtag & Mention Analysis
# ------------------------------------------------
def get_top_hashtags(con, top_n=10, table_name="tweets"):
    """
    Returns the top N hashtags in the dataset based on their frequency of appearance.
    """
    query = f"""
    WITH cleaned AS (
        SELECT regexp_replace(hashtags, '[\[\]"]', '') as clean_hashtags
        FROM {table_name}
        WHERE hashtags IS NOT NULL AND hashtags <> ''
    ),
    split AS (
        SELECT LOWER(TRIM(unnest(string_split(clean_hashtags, ',')))) as hashtag
        FROM cleaned
    )
    SELECT hashtag, COUNT(*) as frequency
    FROM split
    WHERE hashtag <> ''
    GROUP BY hashtag
    ORDER BY frequency DESC
    LIMIT {top_n}
    """
    return con.execute(query).fetchdf()

def get_top_mentions(con, top_n=10, table_name="tweets"):
    """
    Returns the top N mentioned users in the dataset based on frequency.
    """
    query = f"""
    WITH cleaned AS (
        SELECT regexp_replace(mentionedUsers, '[\[\]"]', '') as clean_mentions
        FROM {table_name}
        WHERE mentionedUsers IS NOT NULL AND mentionedUsers <> ''
    ),
    split AS (
        SELECT LOWER(TRIM(unnest(string_split(clean_mentions, ',')))) as mention
        FROM cleaned
    )
    SELECT mention, COUNT(*) as frequency
    FROM split
    WHERE mention <> ''
    GROUP BY mention
    ORDER BY frequency DESC
    LIMIT {top_n}
    """
    return con.execute(query).fetchdf()


# ------------------------------------------------
# 8. Random Sample of Tweets
# ------------------------------------------------
def get_random_tweets(con, table_name="tweets", limit=5):
    """
    Returns a random sample of tweets for manual inspection.
    """
    query = f"""
    SELECT 
        id,
        user,
        text,
        replyCount,
        retweetCount,
        likeCount,
        viewCount,
        date
    FROM {table_name}
    ORDER BY RANDOM()
    LIMIT {limit}
    """
    return con.execute(query).fetchdf()

def print_random_tweets_sample(con, table_name="tweets", limit=5):
    """
    Prints a random sample of tweets. 
    """
    df_sample = get_random_tweets(con, table_name, limit)
    for i, row in df_sample.iterrows():
        print(f"\n--- Tweet {i+1} ---")
        print(f"ID:      {row['id']}")
        print(f"User:    {row['user']}")
        print(f"Date:    {row['date']}")
        print(f"Text:    {str(row['text'])[:200]} ...")  # truncated
        print(f"Replies: {row['replyCount']}, Retweets: {row['retweetCount']}, Likes: {row['likeCount']}, Views: {row['viewCount']}")

# ------------------------------------------------
# 9. Comprehensive Analysis Function
# ------------------------------------------------
def analyze_database(db_path="database/tweets.duckdb", table_name="tweets"):
    """
    Connects to the DuckDB, runs a series of analysis queries, and prints the results.
    """
    con = connect_to_database(db_path)
    
    print("1) Table Overview")
    info_df = get_table_info(con, table_name)
    total_rows = get_total_row_count(con, table_name)
    print(f"Total rows: {total_rows:,}")
    display(info_df)  # In a Jupyter notebook, display as a DataFrame
    
    print("\n2) Null/Empty Value Analysis")
    print_null_summary(con, table_name)
    
    print("\n3) Basic Numeric Statistics")
    print_basic_numeric_stats(con, table_name)
    
    print("\n4) Top Tweets by Likes")
    top_likes = get_top_tweets_by_metric(con, metric="likeCount", top_n=5, table_name=table_name)
    display(top_likes)
    
    print("\n5) Top Users by Tweet Count")
    top_users = get_top_users_by_tweet_count(con, top_n=5, table_name=table_name)
    display(top_users)
    
    print("\n6) Top Hashtags")
    top_hashtags = get_top_hashtags(con, top_n=5, table_name=table_name)
    display(top_hashtags)
    
    print("\n7) Top Mentions")
    top_mentions = get_top_mentions(con, top_n=5, table_name=table_name)
    display(top_mentions)

    print("\n9) Random Sample of Tweets")
    print_random_tweets_sample(con, table_name, limit=5)
    
    con.close()

analyze_database("nl2sql_agent/database/tweets.duckdb", "tweets")



1) Table Overview
Total rows: 20,000,016


,column_name,column_type,null,key,default,extra,distinct_count
0,id,VARCHAR,YES,None,None,None,19931525
1,text,VARCHAR,YES,None,None,None,19112828
2,url,VARCHAR,YES,None,None,None,19824130
3,epoch,VARCHAR,YES,None,None,None,3775439
4,media,VARCHAR,YES,None,None,None,2146688
5,retweetedTweet,VARCHAR,YES,None,None,None,9
6,retweetedTweetID,VARCHAR,YES,None,None,None,28472
7,retweetedUserID,VARCHAR,YES,None,None,None,9446
8,id_str,VARCHAR,YES,None,None,None,19920563
9,lang,VARCHAR,YES,None,None,None,103



2) Null/Empty Value Analysis

Total rows in tweets: 20,000,016

Null/Empty value counts (column : null_count (percentage)):
id                            1 (  0.0%)
text                         33 (  0.0%)
url                          70 (  0.0%)
epoch                        73 (  0.0%)
media                       100 (  0.0%)
retweetedTweet               48 (  0.0%)
retweetedTweetID     19,888,686 ( 99.4%)
retweetedUserID      19,888,668 ( 99.4%)
id_str                       75 (  0.0%)
lang                         95 (  0.0%)
rawContent                  126 (  0.0%)
replyCount                    0 (  0.0%)
retweetCount                  0 (  0.0%)
likeCount                     0 (  0.0%)
quoteCount                    0 (  0.0%)
conversationId              123 (  0.0%)
conversationIdStr           117 (  0.0%)
hashtags                    124 (  0.0%)
mentionedUsers              125 (  0.0%)
links                       125 (  0.0%)
viewCount                     0 (  0.0%)
quotedTweet   

,id,user,text,metric_value
0,https://twitter.com/foutsphotos/status/1814676...,None,[],1.814655e+18
1,24,None,5,9.280610e+17
2,2,None,0,5.824521e+08
3,0,None,0,2.467791e+06
4,1813209070288920669,"{'id': 943974347546521600, 'id_str': '94397434...",i thought that was Donald Trump,1.085449e+06



5) Top Users by Tweet Count


...


6) Top Hashtags


,hashtag,frequency
0,],17892014
1,'text': 'maga'}],414223
2,{'indices': [0,275093
3,'text': 'trump2024'}],215480
4,'text': 'maga'},131943



7) Top Mentions


,mention,frequency
0,'indices': [0,13288690
1,],5695768
2,13]}],1321669
3,12]}],1121702
4,11]}],1100256



9) Random Sample of Tweets

--- Tweet 1 ---
ID:      1814781025559498768
User:    {'id': 1581699253218451460, 'id_str': '1581699253218451460', 'url': 'https://twitter.com/Baby', 'username': 'Baby', 'rawDescription': 'Hello My Name is Baby and I approve this message. it seems that î’ve been away too long…', 'created': datetime.datetime(2022, 10, 16, 17, 31, 34, tzinfo=datetime.timezone.utc), 'followersCount': 93, 'friendsCount': 260, 'statusesCount': 685, 'favouritesCount': 3719, 'listedCount': 0, 'mediaCount': 479, 'location': 'Paris, France', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1581700032876761097/z5omCIP2_normal.jpg', 'profileBannerUrl': 'PW', 'protected': 'PW', 'verified': False, 'blue': False, 'blueType': None, 'descriptionLinks': ['PW'], '_type': 'PW'}
Date:    None
Text:    We love you, Joe but you have to realize that four more years at this pace is simply not feasible and make the right decision : withdraw from a race we are all as human beings about to lo

In [7]:
def run_query(query, db_path="nl2sql_agent/database/tweets.duckdb"):
    """
    Execute a SQL query against the DuckDB database and return results as a pandas DataFrame.
    
    Args:
        query (str): SQL query to execute
        db_path (str): Path to the DuckDB database file
        
    Returns:
        pandas.DataFrame: Query results
    """
    with duckdb.connect(db_path) as con:
        return con.execute(query).fetchdf()
    
query = "SELECT lang, SUM(likeCount) AS totalLikes FROM tweets GROUP BY lang ORDER BY totalLikes DESC LIMIT 5"
results = run_query(query)

In [8]:
results

,lang,totalLikes
0,0,1.814655e+18
1,"{'count': '146', 'state': 'EnabledWithCount'}",9.280610e+17
2,"{'count': '35', 'state': 'EnabledWithCount'}",5.824521e+08
3,en,4.917036e+08
4,es,9.732924e+06


In [13]:
def investigate_lang_values(db_path="database/tweets.duckdb"):
    """
    Investigates potentially problematic language values in the dataset.
    """
    query = """
    SELECT 
        CASE 
            WHEN lang IS NULL THEN 'NULL'
            WHEN lang = '' THEN 'EMPTY'
            ELSE lang 
        END as lang_value,
        COUNT(*) as count,
        SUM(likeCount) as total_likes
    FROM tweets 
    GROUP BY lang
    ORDER BY count DESC
    LIMIT 10
    """
    
    results = run_query(query, db_path)
    print("\nLanguage value analysis:")
    print("-" * 50)
    for _, row in results.iterrows():
        print(f"Value: {str(row['lang_value']):<40} Count: {row['count']:,}")
        print(f"Total Likes: {row['total_likes']:,}")
        print("-" * 50)

#### Unique Languages in the data and issues related to the column

In [14]:
print_unique_langs()


Found 103 unique language values:
----------------------------------------
Language: en              Count: 17,451,496
Language: qme             Count: 692,539
Language: es              Count: 388,020
Language: und             Count: 139,340
Language: fr              Count: 136,226
Language: pt              Count: 135,501
Language: et              Count: 107,133
Language: qht             Count: 95,801
Language: de              Count: 88,517
Language: ja              Count: 85,249
Language: it              Count: 64,833
Language: tr              Count: 63,103
Language: tl              Count: 62,417
Language: eu              Count: 59,243
Language: zxx             Count: 56,794
Language: qam             Count: 45,304
Language: in              Count: 45,236
Language: nl              Count: 41,884
Language: pl              Count: 23,988
Language: art             Count: 19,025
Language: no              Count: 18,850
Language: ht              Count: 16,183
Language: ca              Count: 1

## Sample from raw data for preliminary analysis

In [6]:
def sample_partition():
    import pandas as pd
    import glob
    import os
    
    # Get the first CSV.GZ file from part_1
    pattern = os.path.join("sample_data/part_1", "*.csv.gz")
    files = glob.glob(pattern)
    
    if not files:
        raise FileNotFoundError("No CSV.GZ files found in sample_data/part_1")
    
    # Take the first file found
    file_path = files[0]
    print(f"Sampling from: {file_path}")
    
    # Read 1000 random rows
    # Use chunksize for memory efficiency when dealing with large files
    df = pd.read_csv(file_path, compression='gzip', nrows=100000)
    
    return df

# sample_df = sample_partition()
# print("\nSample shape:", sample_df.shape)
# print("\nSample columns:", sample_df.columns.tolist())
# print("\nFirst few rows:")
# print(sample_df.head())

In [7]:
import os 

def sample_and_save_partition(output_path="sample_data/sampled_tweets_b.csv"):
    """
    Sample 1000 rows from a partition and save to CSV.
    
    Args:
        output_path (str): Path where the sampled CSV will be saved
    """
    # Get the sample
    df = sample_partition()
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"\nSaved {len(df)} rows to: {output_path}")
    
    return df

# Example usage:
if __name__ == "__main__":
    df = sample_and_save_partition()

Sampling from: sample_data/part_1/may_july_chunk_11.csv.gz

Saved 50000 rows to: sample_data/sampled_tweets_b.csv


In [4]:
def analyze_columns(df):
    """
    Analyze each column in the dataframe and provide detailed information about data types,
    sample values, and basic statistics.
    
    Args:
        df (pd.DataFrame): Input dataframe to analyze
        
    Returns:
        dict: Dictionary containing analysis results for each column
    """
    analysis = {}
    
    for column in df.columns:
        # Initialize column analysis
        col_data = {
            'dtype': str(df[column].dtype),
            'null_count': df[column].isna().sum(),
            'null_percentage': (df[column].isna().sum() / len(df)) * 100,
            'unique_count': df[column].nunique(),
            'sample_values': df[column].dropna().head(3).tolist()
        }
        
        # Add type-specific analysis
        if df[column].dtype in ['int64', 'float64']:
            col_data.update({
                'min': df[column].min(),
                'max': df[column].max(),
                'mean': df[column].mean(),
                'median': df[column].median()
            })
        elif df[column].dtype == 'object':
            # For string/object columns
            non_null_values = df[column].dropna()
            if len(non_null_values) > 0:
                col_data.update({
                    'avg_length': non_null_values.str.len().mean(),
                    'max_length': non_null_values.str.len().max(),
                    'contains_numbers': any(str(x).replace('.','',1).isdigit() 
                                         for x in non_null_values.head(100)),
                    'most_common': df[column].value_counts().head(3).to_dict()
                })
        
        analysis[column] = col_data
    
    return analysis

def print_column_analysis(df):
    """
    Print a formatted analysis of all columns in the dataframe.
    
    Args:
        df (pd.DataFrame): Input dataframe to analyze
    """
    analysis = analyze_columns(df)
    
    print("\nColumn Analysis Report")
    print("=" * 80)
    
    for column, data in analysis.items():
        print(f"\nColumn: {column}")
        print("-" * 40)
        print(f"Data Type: {data['dtype']}")
        print(f"Null Count: {data['null_count']} ({data['null_percentage']:.2f}%)")
        print(f"Unique Values: {data['unique_count']}")
        print(f"Sample Values: {data['sample_values']}")
        
        # Print numeric statistics if available
        if 'mean' in data:
            print(f"Numeric Statistics:")
            print(f"  Min: {data['min']}")
            print(f"  Max: {data['max']}")
            print(f"  Mean: {data['mean']:.2f}")
            print(f"  Median: {data['median']}")
        
        # Print string statistics if available
        if 'avg_length' in data:
            print(f"String Statistics:")
            print(f"  Average Length: {data['avg_length']:.2f}")
            print(f"  Max Length: {data['max_length']}")
            print(f"  Contains Numbers: {data['contains_numbers']}")
            print(f"  Most Common Values:")
            for value, count in data['most_common'].items():
                print(f"    - {value}: {count} occurrences")

# Example usage:
if __name__ == "__main__":
    df = sample_partition()
    print_column_analysis(df)

Sampling from: sample_data/part_1/may_july_chunk_11.csv.gz

Column Analysis Report

Column: id
----------------------------------------
Data Type: int64
Null Count: 0 (0.00%)
Unique Values: 1000
Sample Values: [1801321015319896315, 1801321014204178511, 1801321014040686814]
Numeric Statistics:
  Min: 1801320226736218583
  Max: 1801321015319896315
  Mean: 1801320659818732544.00
  Median: 1.8013207082866342e+18

Column: text
----------------------------------------
Data Type: object
Null Count: 0 (0.00%)
Unique Values: 998
Sample Values: ['@BidenHQ Thank you, PM Meloni, for shepherding our "lost little POTUS" back to the herd.\nI\'m so sorry you and Macron, etc., have to provide nursing care Biden\nThis is REALLY not in your job description.', '@MuskUniverseUsa 9', '@BidenHQ Lmfao. Biden has fallen down 17 times this year. You’re finding the one video where it was raining and he was wearing dress shoes down a metal ramp 😂😂😂 you guy have nothing on him.  \nGo find the cheer he got from tho

/Users/mahdi/repos/USC_hacketon/nl2sql_agent/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mahdi/repos/USC_hacketon/nl2sql_agent/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mahdi/repos/USC_hacketon/nl2sql_agent/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mahdi/repos/USC_hacketon/nl2sql_agent/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/mahdi/repos/USC_hacketon/nl2sql_agent/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  retur